In [ ]:
# 順張り戦略のお試し
# スリッページが発生しない売買戦略を試す

In [ ]:
from pathlib import Path
import datetime

import matplotlib.pyplot as plt
import polars as pl
import numpy as np

import crypto
import data_fetcher

In [ ]:
start_date = datetime.datetime(2024, 1, 1)
end_date = datetime.datetime(2024, 1, 31)
symbol = "BTC_JPY"
interval = 1

In [ ]:
fetcher = data_fetcher.gmo.GMOFethcer()
df = fetcher.fetch_ohlc(
    symbol, interval=datetime.timedelta(minutes=interval), start_date=start_date, end_date=end_date
)
df = crypto.features.calc_features(df)

In [ ]:
# 移動平均線ストレッチ
window_size = 10
col_key = f"MA_{window_size}"
deviation_rate = 0.001
df = df.with_columns(
    pl.col("close").rolling_mean(window_size=window_size).alias(col_key)
).with_columns(
    ((pl.col("close") - pl.col(col_key)) / pl.col("close") > deviation_rate).shift().alias("sell_point"),
    ((pl.col(col_key) - pl.col("close")) / pl.col("close") > deviation_rate).shift().alias("buy_point"),
    pl.col("close").shift().alias("target_price"),
).with_columns(
    ((pl.col("target_price") > pl.col("low")) & pl.col("buy_point")).alias("buy_executed"),
    ((pl.col("target_price") < pl.col("high")) & pl.col("sell_point")).alias("sell_executed")
)

In [ ]:
profits = crypto.simulate.simulate_long_trade(
    df, "sell_executed", "buy_executed", "target_price", "target_price", 100
)
plt.plot(profits.cumsum())

In [ ]:
# ブレイクアウト
window_size = 30
df = df.with_columns(
    ((pl.col("close").rolling_max(window_size=window_size) - pl.col("close")) < 1e-5).shift().alias("buy_point"),
    ((pl.col("close") - pl.col("close").rolling_min(window_size=window_size)) < 1e-5).shift().alias("sell_point"),
    pl.col("close").shift().alias("target_price"),
).with_columns(
    ((pl.col("target_price") > pl.col("low")) & pl.col("buy_point")).alias("buy_executed"),
    ((pl.col("target_price") < pl.col("high")) & pl.col("sell_point")).alias("sell_executed")
)

In [ ]:
profits = crypto.simulate.simulate_long_trade(
    df, "sell_executed", "buy_executed", "target_price", "target_price", 100
)
plt.plot(profits.cumsum())

In [ ]:
# 連続上昇
